# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [111]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [112]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print (df.head())

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [113]:
# frequency table for prestige and whether or not someone was admitted
print (df.describe())

            admit         gre         gpa    prestige
count  397.000000  397.000000  397.000000  397.000000
mean     0.317380  587.858942    3.392242    2.488665
std      0.466044  115.717787    0.380208    0.947083
min      0.000000  220.000000    2.260000    1.000000
25%      0.000000  520.000000    3.130000    2.000000
50%      0.000000  580.000000    3.400000    2.000000
75%      1.000000  660.000000    3.670000    3.000000
max      1.000000  800.000000    4.000000    4.000000


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [114]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix = 'Prestige')
print (dummy_ranks.head())

   Prestige_1.0  Prestige_2.0  Prestige_3.0  Prestige_4.0
0             0             0             1             0
1             0             0             1             0
2             1             0             0             0
3             0             0             0             1
4             0             0             0             1


In [115]:
cols_to_keep = ['admit', 'gre', 'gpa', 'prestige']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print (data.head())

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


/Users/judenwosu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


#### 2.2 When modeling our class variables, how many do we need? 



Answer: we need 4 variables.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [116]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
print (pd.crosstab(df['admit'], df['prestige'], rownames = ['admit']))

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [139]:
top_ranked_admission = 33/28
print (top_ranked_admission)

1.1785714285714286


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [140]:
non_top_ranked_admission = (53+28+12)/(95+93+55)
print (non_top_ranked_admission)

0.38271604938271603


#### 3.3 Calculate the odds ratio

In [141]:
odds_ratio = (33+53+28+12)/(28+95+93+55)
print (odds_ratio)

0.46494464944649444


#### 3.4 Write this finding in a sentenance: 

Answer: The hand-calculated odds ratio above (0.4649) indicates that the odds of getting accepted into graduate school is 0.465 higher given the institution's higher ranking.

#### 3.5 Print the cross tab for prestige_4

In [142]:
print (pd.crosstab(df[df.prestige==4.0]['admit'], df[df.prestige==4.0]['prestige'], rownames=['admit']))

prestige  4.0
admit        
0          55
1          12


#### 3.6 Calculate the OR 

In [143]:
prestige_4_Odds_Ratio = 12/55
print(prestige_4_Odds_Ratio)

0.21818181818181817


#### 3.7 Write this finding in a sentence

Answer:  Getting accepted into graduate school from a Prestige_4 institution is 0.218 lower given the school ranking compared to top ranked institutions

## Part 4. Analysis

In [121]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print (data.head())

   admit    gre   gpa
0      0  380.0  3.61
1      1  660.0  3.67
2      1  800.0  4.00
3      1  640.0  3.19
4      0  520.0  2.93


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [122]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [127]:
train_cols = data.columns[1:]
logit = sm.Logit(data['admit'], data[train_cols])

#### 4.2 Fit the model

In [128]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.601299
         Iterations 5


#### 4.3 Print the summary results

In [100]:
print (result.summary())

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      394
Method:                           MLE   Df Model:                            2
Date:                Mon, 11 Dec 2017   Pseudo R-squ.:                 0.03773
Time:                        15:49:58   Log-Likelihood:                -238.72
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 8.600e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0026      0.001      2.502      0.012       0.001       0.005
gpa            0.7357      0.320      2.296      0.022       0.108       1.364
intercept     -4.8598      1.078     -4.507      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [101]:
print (np.exp(result.params))

gre          1.002650
gpa          2.086864
intercept    0.007752
dtype: float64


In [102]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print (np.exp(conf))

               2.5%     97.5%        OR
gre        1.000574  1.004730  1.002650
gpa        1.113542  3.910944  2.086864
intercept  0.000937  0.064164  0.007752


#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [132]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = int(n / arrays[0].size)
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [133]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print (gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print (gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [134]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])

/Users/judenwosu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


#### 5.2 Make predictions on the enumerated dataset

In [135]:
combos['admit_pred'] = result.predict(combos[train_cols])

#### 5.3 Interpret findings for the last 4 observations

In [136]:
combos['admit_pred'].tail()

395    0.514427
396    0.549822
397    0.549822
398    0.549822
399    0.549822
Name: admit_pred, dtype: float64

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.